In [1]:
import yaml
import clickhouse_connect
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import importlib

2023-06-24 19:24:46.366889: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-06-24 19:24:46.366915: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [31]:
filename = 'db_credentials'
with open(f'{filename}.yaml','r') as f:
    output = yaml.safe_load(f)
print(output) 

{'host': 'db-access.improvado.io', 'port': 443, 'database': 'im_7690_4c9', 'user': 'im_7690_db', 'password': '4LFh3OgdbOC4HBxcXSsH'}


In [32]:
client = clickhouse_connect.get_client(**output)

In [33]:
all_tables = client.command('SHOW TABLES').split('\n')

In [35]:
','.join(all_tables)

'ads_7690_bing_all_data,ads_7690_dcs_all_data,ads_7690_google_ads_ql_all_data,ads_7690_tiktok_ads_all_data,ads_creative_7690_facebook_all_data,ads_creative_with_destination_url_7690_facebook_all_data,ads_device_7690_google_ads_ql_all_data,ads_geo_7690_tiktok_ads_all_data,ads_placement_sites_7690_google_cm_all_data,ads_placement_sites_geo_7690_google_cm_all_data,ads_target_7690_stack_adapt_all_data,adsets_device_geo_7690_apple_search_all_data,age_extended_7690_google_ads_ql_all_data,age_gender_7690_youtube_organic_all_data,age_gender_by_ads_7690_facebook_all_data,age_gender_marketing_dummydata_youtube_organic_all_data,attribution__biz_attribution_all_models_dummydata,attribution__biz_paid_ads_attribution_all_models_dummydata,audience_age_gender_7690_facebook_pages_all_data,audience_age_gender_7690_instagram_organic_all_data,audience_geo_7690_instagram_organic_all_data,biz_attribution_all_models_dummydata,biz_paid_ads_attribution_all_models_dummydata,campaign_7690_google_ads_ql_all_data,

In [22]:
output['model']

{'model_checkpoint': 'juierror/flan-t5-text2sql-with-schema'}

In [2]:
class QueryInterface:

    def __init__(self):
        '''
        Initialization of model and all arguments
        '''
        pass

    def query(self):
        '''
        query model with a question. extra arguments needed when using specific models
        '''
        pass

    def _init_model(self):
        '''
        Model initialization, with a path of configs
        '''



class HF_LLMQuery(QueryInterface):

    def __init__(self, model_config_path, prompt_style_class, generation_args={}, *args, **kwargs):
        self.model, self.tokenizer = self._init_model(model_config_path)
        self.prompt_style = prompt_style_class()
        self.generation_args = generation_args
        
    def query(self, question, **kwargs):

        prompt = self.prompt_style(question, **kwargs)
        input_ids = self.tokenizer(prompt, return_tensors="pt").input_ids
        
        output = self.model.generate(input_ids=input_ids, **self.generation_args)
        out_text = self.tokenizer.decode(output[0], skip_special_tokens=True)

        return out_text

    def _init_model(self, model_config_path):

        with open(f'{model_config_path}','r') as f:
            configs = yaml.safe_load(f)

        module = importlib.import_module('transformers')

        TokenizerClass = module.__getattr__(configs['TokenizerClass'])
        ModelClass = module.__getattr__(configs['ModelClass'])

        tokenizer = TokenizerClass.from_pretrained(**configs['tokenizer'])
        model = ModelClass.from_pretrained(**configs['model'])

        return model, tokenizer


class SimplePrompt:

    def __init__(self):
        pass
    
    def __call__(self, query, **kwargs):
        return query


class QuestionTableRowsPrompt:

    def __init__(self):
        pass
    
    def __call__(self, query, tables, rows, **kwargs):

        tables, rows = self.prepare(tables, rows)
        res = """question: {} {}: {}""".format(
                                        query,
                                        ', '.join(tables),
                                        ', '.join(rows)
                                        )
        return res
    
    def prepare(self, tables, rows):

        if isinstance(tables, str):
            tables = [tables]
        if isinstance(rows, str):
            rows = [rows]


        assert isinstance(tables, list), 'Passed tables are not List or Str'
        assert isinstance(rows, list), 'Passed rows are not List or Str'
        
        return tables, rows
    

In [57]:
query  = 'get people name with age equal 25'
tables = ['table']
rows = ['id', 'name', 'age']

res = """question: {} {}: {}""".format(
                                        query,
                                        ', '.join(tables),
                                        ', '.join(rows)
                                        )

In [58]:
res

'question: get people name with age equal 25 table: id, name, age'

In [40]:
', '.join(['table', 'table 2'])

'table, table 2'

In [3]:
querier = HF_LLMQuery('model_config.yaml', QuestionTableRowsPrompt)

In [5]:
ans = querier.query('get people name with age equal 25', tables='my_table', rows = ['id', 'name', 'age'])

/home/victor/anaconda3/lib/python3.7/site-packages/transformers/generation/utils.py:1392: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


In [6]:
querier.tokenizer.decode(ans[0], skip_special_tokens=True)

'SELECT name FROM table WHERE age = 25'

In [4]:
import openai
import tiktoken


ModuleNotFoundError: No module named 'tiktoken'

In [ ]:
!pip install tiktoken

: 

In [ ]:
openai.api_key = "token"


class OpenAI_LLMQuery(QueryInterface):

    def __init__(self, model_config_path, prompt_style_class, generation_args={}, *args, **kwargs):
        self.model_configs = self._init_model(model_config_path)
        self.prompt_style = prompt_style_class()
        self.generation_args = generation_args

    def query(self, question, **kwargs):

        prompt = self.prompt_style(question, **kwargs)
        input_ids = self.tokenizer(prompt, return_tensors="pt").input_ids
        
        output = self.model.generate(input_ids=input_ids, **self.generation_args)
        out_text = self.tokenizer.decode(output[0], skip_special_tokens=True)

        return out_text

    def _init_model(self, model_config_path):

         with open(f'{model_config_path}','r') as f:
            configs = yaml.safe_load(f)

        module = importlib.import_module('openai')

        return configs

    @sleep_and_retry
    @limits(calls=3, period=62)
    def get_answer(text: str) -> str:
        text = cut_text(text)
        completion = openai.ChatCompletion.create(
            temperature=0,
            max_tokens=COMPLETION_TOKENS,
            top_p=1.0,
            frequency_penalty=0.0,
            presence_penalty=0.0,
            model=MODEL_NAME,
            messages=[
                {
                    "role": "system",
                    "content": role,
                },
                {"role": "user", "content": promt},
                {"role": "user", "content": few_shot},
                {"role": "user", "content": "Текст новости: " + text},
                {"role": "user", "content": answer},
            ],
        )
        return str(completion["choices"][0]["message"]["content"])

    

MODEL_NAME = "gpt-3.5-turbo"
MAX_TOKENS = 4096
NUM_SAFE_TOKENS = 50
COMPLETION_TOKENS = 128

role = "Действуй как социолог, который хорошо разбирается в приемах манипуляции человеком через новостные тексты"
promt = """Манипулятивность, стремление скрыто воздействовать на адресата, является характерной чертой медиатекстов. Она типична для СМИ разной направленности (проправительственных, независимых и оппозиционных).
Манипуляция информацией заключается в ее искажении, селекции и умолчании. """
few_shot = """Текст новости: Одно из самых поразительных (и притом разрушительных) проявлений разложения и деградации российской власти — ее патологическое в своей безграмотности самодурство; Но не исправлять же текст указа!
Ясно, что это обнажит пусть перед ничтожно малой, но все же частью народа истинный уровень — причем не 40 одного только Медведева, но и всей правящей Россией тусовки
Текст содержит манипуляции? 
Ответ: Да

Текст новости: 'Сборная России победила команду Литвы на чемпионате Европы по гандболу. ТАСС, 13 января. Сборная России по гандболу со счетом 29:27 обыграла команду Литвы в матче первого тура предварительного раунда чемпионата Европы. Встреча прошла в Словакии.
Сборные России и Литвы выступают в группе F вместе с командами Словакии и Норвегии, которые сыграют позднее в четверг. Следующий матч россияне проведут 15 января против норвежцев, а литовцы в этот же день встретятся со словаками. Две лучшие команды группы выйдут в следующий раунд. Чемпионат Европы завершится 30 января.'
Текст содержит манипуляции? 
Ответ: Нет"""
text_prefix = "Текст новости: "
answer = """Текст содержит манипуляции? 
Ответ:"""

encoding = tiktoken.encoding_for_model(MODEL_NAME)
used_tokens = (
    len(encoding.encode(role))
    + len(encoding.encode(promt))
    + len(encoding.encode(few_shot))
    + len(encoding.encode(text_prefix))
    + len(encoding.encode(answer))
)


def cut_text(text: str) -> str:
    text_tokens = encoding.encode(text)
    if used_tokens + len(text_tokens) < MAX_TOKENS:
        return text
    else:
        return encoding.decode(
            text_tokens[: MAX_TOKENS - used_tokens - NUM_SAFE_TOKENS - COMPLETION_TOKENS]
        )




In [6]:
module = importlib.import_module('transformers')

In [7]:
AutoTokenizer = module.__getattr__('AutoTokenizer')

In [20]:
tokenizer = AutoTokenizer.from_pretrained("juierror/flan-t5-text2sql-with-schema")
model = AutoModelForSeq2SeqLM.from_pretrained("juierror/flan-t5-text2sql-with-schema")

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

KeyboardInterrupt: 

2992

In [51]:
query = 'SELECT * FROM country_region_mapping LIMIT 5'
res = client.command(query)


In [58]:
import signal

class TimeoutException(Exception):
    pass

def timeout_handler(signum, frame):
    raise TimeoutException("Code execution timed out")

def run_with_timeout(func, timeout_seconds):
    # Set the signal handler for the timeout
    signal.signal(signal.SIGALRM, timeout_handler)
    signal.alarm(timeout_seconds)  # Set the timeout

    try:
        result = func()  # Execute the code
        signal.alarm(0)  # Reset the alarm if the code completes within the timeout
        return result
    except TimeoutException:
        print("Code execution timed out")
        return None

# Example usage
def long_running_code():
    # Your long-running code goes here
    query = 'SELECT * FROM web_analytics__utm_quality'
    res = client.command(query)
    return res

timeout = 1  # Set the desired timeout in seconds
result = run_with_timeout(long_running_code, timeout)

Code execution timed out


In [59]:
result

In [60]:
result

In [63]:
query = """SELECT column_name FROM information_schema.columns WHERE table_name = 'country_region_mapping' """
res = client.command(query)

In [65]:
res.split('\n')

['country_name', 'country_code', 'region_name', 'region_code']

In [4]:
query = 'SELECT *'
result = client.execute(query)

# Process the query result as needed
for row in result:
    print(row)

UnexpectedPacketFromServerError: Code: 102. Unexpected packet from server db-access.improvado.io:443 (expected Hello or Exception, got Unknown packet)